In [ ]:
import keras
import numpy as np
import tensorflow as tf
import sklearn

from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K

### Import Test Image

In [ ]:
img = keras.preprocessing.image.load_img('test4/train/ga_050/ga_050_1.jpg')
x = keras.preprocessing.image.img_to_array(img)
x.shape

### Build the Binary Model

In [ ]:
# dimensions of our images.
img_width, img_height =480, 640
#tgt_width, tgt_height = 150, 150

train_data_dir = 'test3/train'
validation_data_dir = 'test3/validation'
nb_train_samples = 2000
nb_validation_samples = 200
epochs = 2
batch_size = 10

if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)

model = Sequential()

model.add(Conv2D(32, (3, 3), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))


model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

"""
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
"""

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
#Use softmax for the multi-class problem
#model.add(Activation('softmax'))
model.add(Activation('sigmoid'))

#use loss='categorical_crossentropy'
model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(rescale=1. / 255)
#    rescale=1. / 255,
#    shear_range=0.2,
#    zoom_range=0.2,
#    width_shift_range=0.2,
#    height_shift_range=0.2)
#    horizontal_flip=True)

# this is the augmentation configuration we will use for testing:
# only rescaling

test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    #use class_mode='categorical'
    class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    #use class_mode='categorical'
    class_mode='binary')

model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples // batch_size)

model.save_weights('test3_model')

### Train a Multiclass Model

In [ ]:
# dimensions of our images.
img_width, img_height =480, 640
tgt_width, tgt_height = int(img_width/2), int(img_height/2)

train_data_dir = 'test4/train'
validation_data_dir = 'test4/validation'
nb_train_samples = 3000
nb_validation_samples = 300
num_classes = 3
epochs = 5
batch_size = 30

if K.image_data_format() == 'channels_first':
    input_shape = (3, tgt_width, tgt_height)
else:
    input_shape = (tgt_width, tgt_height, 3)

model = Sequential()

model.add(Conv2D(32, (1, 1), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))


model.add(Conv2D(32, (1, 1)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))


model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))


model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.3))
model.add(Dense(num_classes))
#Use softmax for the multi-class problem
model.add(Activation('softmax'))
#model.add(Activation('sigmoid'))

#use loss='categorical_crossentropy'
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(rescale=1. / 255)
#    rescale=1. / 255,
#    shear_range=0.2,
#    zoom_range=0.2,
#    width_shift_range=0.2,
#    height_shift_range=0.2)
#    horizontal_flip=True)

# this is the augmentation configuration we will use for testing:
# only rescaling

test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(tgt_width, tgt_height),
    batch_size=batch_size,
    #use class_mode='categorical'
    class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(tgt_width, tgt_height),
    batch_size=batch_size,
    #use class_mode='categorical'
    class_mode='categorical')

model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples // batch_size)

#Save the model and weights for reuse later
model_json = model.to_json()
with open("model4.json", "w") as json_file:
    json_file.write(model_json)
model.save_weights("test4_model.h5")
print("Saved model to disk")